In [1]:
import functools, operator, requests, os, json
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import gradio as gr

In [2]:
# Set environment variables

import os
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")

os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ["AZURE_OPENAI_ENDPOINT_SCUS"]
os.environ["AZURE_OPENAI_API_KEY"]  = os.environ["AZURE_OPENAI_API_KEY_SCUS"]
os.environ["OPENAI_API_VERSION"]    = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["AZURE_OPENAI_API_TYPE"] = os.environ["OPENAI_API_TYPE"]

MODEL = os.environ["GPT4-0125PREVIEW-128k"]

# https://smith.langchain.com/
os.environ["LANGCHAIN_TRACING_V2"]  = "true"
os.environ["LANGCHAIN_PROJECT"]     = "LangGraph Research Agents"
os.environ["LANGCHAIN_API_KEY"]     = os.environ["LANGCHAIN_API_KEY"]

In [3]:
# Initialize model

llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=1000)

In [4]:
# 1. Define custom tools
@tool("internet_search_tool", return_direct=False) # this is the name tracked in LangSmith
def internet_search_function(query: str) -> str:
    """Searches the internet using DuckDuckGo."""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=5)]
        return results if results else "No results found."

@tool("process_content", return_direct=False)
def process_content_function(url: str) -> str:
    """Processes content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

tools = [internet_search_function, process_content_function]

# AGENTS

## Create an Agent step by step

In [5]:
system_prompt = (
    "system",
    "You are a web searcher. Search the internet for information.")

mp1 = MessagesPlaceholder(variable_name="messages")

mp2 = MessagesPlaceholder(variable_name="agent_scratchpad")

print(system_prompt, mp1, mp2, sep='\n')

('system', 'You are a web searcher. Search the internet for information.')
variable_name='messages'
variable_name='agent_scratchpad'


In [6]:
cpt = ChatPromptTemplate.from_messages([
        system_prompt,
        mp1,
        mp2
    ])

cpt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a web searcher. Search the internet for information.')), MessagesPlaceholder(variable_name='messages'), MessagesPlaceholder(variable_name='agent_scratchpad')])

In [7]:
agent = create_openai_tools_agent(llm, tools, cpt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a web searcher. Search the internet for information.')), MessagesPlaceholder(v

In [8]:
executor = AgentExecutor(agent=agent, tools=tools)
executor

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a web searcher. Search t

## Create an Agent Executor with a Helper Function

In [9]:
# Helper function for creating agents
def create_agent_executor(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [10]:
# Create the Search Agent Executor

search_agent_exec = create_agent_executor(
    llm=llm,
    tools=tools,
    system_prompt="You are a web searcher. Search the internet for information.",
    )

search_agent_exec

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a web searcher. Search t

In [11]:
# Create the Insight Research Agent Executor

insights_research_agent_exec = create_agent_executor(
    llm = llm,
    tools = tools, 
    system_prompt = """You are a Insight Researcher. Do step by step. 
        Based on the provided content first identify the list of topics,
        then search internet for each topic one by one
        and finally find insights for each topic one by one.
        Include the insights and sources in the final response""")

insights_research_agent_exec

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a Insight Researcher. Do

# NODES

## What is a "HumanMessage"

In [12]:
# it's not used in our code directly, but let's see what a HumanMessage is

from langchain_core.messages import HumanMessage

HumanMessage(content="This is the result", name="Node name")

HumanMessage(content='This is the result', name='Node name')

## "agent_node" function to convert the agent response to a human message

In [13]:
# This function facilitates creating nodes in the graph:
# it takes care of converting the agent response to a human message. 
# This is important because that is how we will add it the global state of the graph

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

## Create the "Web_Searcher" Node for Searcher Agents Executor

In [14]:
# create the search node, starting from the search agent executor

search_node = functools.partial(agent_node, agent=insights_research_agent_exec, name="Web_Searcher")
search_node

functools.partial(<function agent_node at 0x7f7ef4b66ac0>, agent=AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTempla

## Create the "Insight_Researcher" Node for Insights_Researcher Agents Executor

In [15]:
# create the insights_research node, starting from the insights_research agent executor

insights_research_node = functools.partial(agent_node, agent=insights_research_agent_exec, name="Insight_Researcher")
insights_research_node

functools.partial(<function agent_node at 0x7f7ef4b66ac0>, agent=AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTempla

## Create the "supervisor" Node 
Let's recall that:
- we do not have a supervisor agent
- the current nodes are **Web_Search** and **Insight_Researcher**
- a "node" typically include the following components:
```
functool.partial(
...
| ChatPromptTemplate
| RunnableBinding
| OpenAIToolsAgentOutputParser
)
```

### Prepare the variables

In [16]:
# given the two nodes...
members = ["Web_Searcher", "Insight_Researcher"]

# ...the possible chancesfor supervisor include "FINISH"
options = ["FINISH"] + members

print(f"members: {members},\noptions: {options}")

members: ['Web_Searcher', 'Insight_Researcher'],
options: ['FINISH', 'Web_Searcher', 'Insight_Researcher']


### Create the ChatPromptTemplate object

The process is the same used for the other two nodes, but in this case we also use a **partial function**.<br/>

A **partial function*** is a function that is created by fixing a certain number of arguments of an existing function. This is achieved using the functools.partial function, which allows you to set default values for one or more arguments of a function, effectively creating a new function with those default values already set.
The functools.partial function is used to create partial functions in Python. It allows you to fix a certain number of arguments of an existing function, effectively creating a new function with those fixed arguments. This can be useful when you want to create a simplified version of a function with some arguments pre-filled, or when you need to adapt a function to be used as a callback with a different signature.

By using **functools.partial**, you can create specialized functions from more general ones, making your code more modular and easier to maintain. This can be especially handy when working with libraries or frameworks that require functions with specific signatures.

In [17]:
system_prompt_supervisor = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.")

mp1 = MessagesPlaceholder(variable_name="messages")

mp2 = (
    "system",
    "Given the conversation above, who should act next?"
    " Or should we FINISH? Select one of: {options}")

cpt_supervisor = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_supervisor),
        mp1,
        mp2
    ]
).partial(options=str(options), members=", ".join(members))

cpt_supervisor

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Web_Searcher', 'Insight_Researcher']", 'members': 'Web_Searcher, Insight_Researcher'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['members'], template='You are a supervisor tasked with managing a conversation between the following workers: {members}. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['options

In [18]:
# Using openai function calling can make output parsing easier for us

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

tools = [function_def]

tools

[{'name': 'route',
  'description': 'Select the next role.',
  'parameters': {'title': 'routeSchema',
   'type': 'object',
   'properties': {'next': {'title': 'Next',
     'anyOf': [{'enum': ['FINISH', 'Web_Searcher', 'Insight_Researcher']}]}},
   'required': ['next']}}]

In [19]:
# As we saw for the two nodes above, each node include a "RunnableBinding" object
# This object "binds" the function definition to the function name "route"

rb = llm.bind_functions(functions=tools, function_call="route")
rb

RunnableBinding(bound=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f7ef4cd1990>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f7ef4b04950>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1000, azure_endpoint='https://mmopenaiscus.openai.azure.com/', deployment_name='gpt4-0125preview-128k', openai_api_version='2024-02-15-preview', openai_api_type='azure'), kwargs={'functions': [{'name': 'route', 'description': 'Select the next role.', 'parameters': {'title': 'routeSchema', 'type': 'object', 'properties': {'next': {'title': 'Next', 'anyOf': [{'enum': ['FINISH', 'Web_Searcher', 'Insight_Researcher']}]}}, 'required': ['next']}}], 'function_call': {'name': 'route'}})

In [20]:
# We're ready to put together the three elements needed to create the node as a pipeline:
# - ChatPromptTemplate
# - RunnableBinding
# - JsonOutputFunctionsParser


supervisor_chain = (
    cpt_supervisor
    | rb
    | JsonOutputFunctionsParser()
   
)

supervisor_chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Web_Searcher', 'Insight_Researcher']", 'members': 'Web_Searcher, Insight_Researcher'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['members'], template='You are a supervisor tasked with managing a conversation between the following workers: {members}. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['options

# GRAPH!

Now that we have Tools, Agents and Nodes we are ready to build the graph

In [21]:
# First, we need to define the "AgentState", which represents the conversation history

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

In [22]:
# Then, we create the graph using from langgraph.graph import StateGraph

workflow = StateGraph(AgentState)

In [23]:
# To start building the workflow, we add the three nodes

workflow.add_node("Web_Searcher", search_node)
workflow.add_node("Insight_Researcher", insights_research_node)
workflow.add_node("Supervisor", supervisor_chain)

[n for n in workflow.nodes]

['Web_Searcher', 'Insight_Researcher', 'Supervisor']

In [24]:
# Now we add the edges between the two node members and the supervisor
# We want our workers to ALWAYS "report back" to the Supervisor node when done

for member in members:
    print(f"connecting {member} node to the supervisor node...")
    workflow.add_edge(member, "Supervisor")
    
[e for e in workflow.edges]

connecting Web_Searcher node to the supervisor node...
connecting Insight_Researcher node to the supervisor node...


[('Web_Searcher', 'Supervisor'), ('Insight_Researcher', 'Supervisor')]

In [25]:
# The conditional map illustrates the match between each node and its direction

conditional_map = {k: k for k in members}

conditional_map["FINISH"] = END

conditional_map

{'Web_Searcher': 'Web_Searcher',
 'Insight_Researcher': 'Insight_Researcher',
 'FINISH': '__end__'}

In [26]:
# Associate the conditional map to the workflow

workflow.add_conditional_edges(
    "Supervisor", 
    lambda x: x["next"], 
    conditional_map)

In [27]:
# Add entry point

workflow.set_entry_point("Supervisor")

In [29]:
# Compile the graph

graph = workflow.compile()

graph

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<messages,next>(writes=[ChannelWriteEntry(channel='messages', value=_get_state_key(), skip_none=False), ChannelWriteEntry(channel='next', value=_get_state_key(), skip_none=False)]), ChannelWrite<start:Supervisor>(writes=[ChannelWriteEntry(channel='start:Supervisor', value='__start__', skip_none=False)])]), 'Web_Searcher': PregelNode(config={'tags': []}, channels={'messages': 'messages', 'next': 'next'}, triggers=['branch:Supervisor:condition:Web_Searcher'], mapper=functools.partial(<function _coerce_state at 0x7f7ef7c0d940>, <class '__main__.AgentState'>), writers=[ChannelWrite<Web_Searcher,messages,next>(writes=[ChannelWriteEntry(channel='Web_Searcher', value=None, skip_none=False), ChannelWriteEntry(channel='messages', value=_get_state_key(), skip_none=False), ChannelWriteEntry(channel='next', value=_get_state_key(), skip_none=F

In [30]:
# Run the graph

for s in graph.stream({
    "messages": [HumanMessage(content="""Search for the latest AI technology trends in 2024,
            summarize the content. After summarise pass it on to insight researcher
            to provide insights for each topic""")]
}):
    if "__end__" not in s:
        print(s)
        print("----")

{'Supervisor': {'next': 'Web_Searcher'}}
----
{'Web_Searcher': {'messages': [HumanMessage(content="Based on the processed content from various sources, here are the latest AI technology trends for 2024:\n\n1. **Customized Chatbots**: Companies like Google and OpenAI are developing platforms that allow people to customize powerful language models to create their own mini chatbots tailored to specific needs, without requiring coding skills. This democratization of AI tool creation is expected to make generative AI more accessible and useful for the regular, non-tech person.\n\n2. **Generative AI's Second Wave - Video**: Following the explosion of generative models for creating photorealistic images, the next frontier is text-to-video. Companies are rapidly improving the quality of generative video models, with applications ranging from movie production enhancements to marketing and training purposes. This trend is also raising questions about the impact on traditional filmmaking and the 

{'Supervisor': {'next': 'FINISH'}}
----
